food2 의 데이터를 가지고 버트 미사용 코사인, 유클리디안 / bert 사용 코사인, 유클리디안

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/MachineLearningPractice/Colab-Notebooks/FarmKit/foodfile/food2.csv')
df = df.dropna()
df.shape

(691, 4)

In [ ]:
df.head()

,구분,음식명,영문,음식설명
0,면,가락국수,Thick Noodles in Clear Broth,"Noodles in a clear broth, topped with vegetabl..."
1,떡,가래떡,Cylindrical Rice Pasta,Cylindrical rice pasta sliced diagonally into ...
2,찜,가오리찜,Steamed Stingray,Marinated stingray steamed and served with thi...
3,구이,가자미구이,Grilled Sole,"Sole seasoned with salt then grilled, often to..."
4,젓갈,가자미식해,Spicy Fermented Sole,A spicy dish of sole mixed with cooked millet ...


In [ ]:
df.reset_index(inplace=True)
df.drop('index', inplace=True, axis=1)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(stop_words='english')     # 크게 의미없는 단어들을 제외해준다.
tfidf_matrix = tfidf.fit_transform(df['음식설명'])
tfidf_matrix.shape

(691, 1252)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances

cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
distance_sim = euclidean_distances(tfidf_matrix, tfidf_matrix)
print(cosine_sim.shape, distance_sim.shape)

(691, 691) (691, 691)


In [ ]:
!pip install transformers
import torch
from transformers import AutoModel, AutoTokenizer

MODEL_NAME = "bert-base-multilingual-cased"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModel.from_pretrained(MODEL_NAME)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
# [CLS] token을 얻기 위한 함수
def get_cls_token(sent_A):
    model.eval()
    tokenized_sent = tokenizer(
            sent_A,
            return_tensors="pt",
            truncation=True,
            add_special_tokens=True,
            max_length=128
    )
    with torch.no_grad():# 그라디엔트 계산 비활성화
        outputs = model(    # **tokenized_sent
            input_ids=tokenized_sent['input_ids'],
            attention_mask=tokenized_sent['attention_mask'],
            token_type_ids=tokenized_sent['token_type_ids']
            )
    logits = outputs.last_hidden_state[:,0,:].detach().cpu().numpy()
    return logits

In [ ]:
def cls_token_array(df, column) :
  dataset_cls_hidden = []
  for i in range(df.shape[0]):
      q_cls = get_cls_token(str(df.loc[i, column])) #column 예: '음식설명', '영문'
      dataset_cls_hidden.append(q_cls)
  dataset_cls_hidden = np.array(dataset_cls_hidden).squeeze(axis=1)
  #print(dataset_cls_hidden)   
  print(dataset_cls_hidden.shape) # column에 대한 [CLS] 토큰 벡터
  return dataset_cls_hidden

column = '음식설명'
dataset_cls_hidden = cls_token_array(df, column)

(691, 768)


In [ ]:
def to_dataframe (cos, euc, bert_cos, bert_euc):
    df = pd.DataFrame()
    df['cos'] = cos
    df['euc'] = euc
    df['bert cos'] = bert_cos
    df['bert euc'] = bert_euc
    return df

In [ ]:
def bert_food_recommendation(df, dataset_cls_hidden, column, food, similarity): 
  
  index = df[df['음식명'] == food].index[0]
  food_cls_hidden = get_cls_token(str(df.loc[index, column]))

  sim = similarity(food_cls_hidden, dataset_cls_hidden)
  #print(cos_sim)
  if similarity == cosine_similarity:
    top = np.argpartition(sim, -10)[:, -10:] # 상위 10개 인덱스 추출
  else :
    top = np.argpartition(sim, 10)[:, :10]

  return list(df.iloc[top[0]]['음식명'])

In [ ]:
indices = pd.Series(df.index, index=df['음식명']).drop_duplicates()    # 중복 제거

In [ ]:
# 음식명을 입력받으면 코사인 유사도를 통해서 가장 유사도가 높은 상위 10개의 음식 목록 반환
def get_recommendations(title, sim, reverse):
  
  # 음식명을 통해서 전체 데이터 기준 그 음식의 index 값을 얻기
  idx = indices[title]

  # 유사도 매트릭스 에서 idx 에 해당하는 데이터를 (idx, 유사도) 형태로 얻기
  sim_scores = list(enumerate(sim[idx]))

  # 유사도 기준 정렬
  sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=reverse)    # 코사인 유사도는 True, 유클리디안 유사도는 False를 넣어줘야함

  # 자기 자신을 제외한 10개의 추천 음식을 슬라이싱
  sim_scores = sim_scores[1:11]

  # 추천 음식목록 10개의 인덱스 정보 추출
  movie_indices = [i[0] for i in sim_scores]

  # 인덱스 정보를 통해 음식명 추출
  return list(df['음식명'].iloc[movie_indices])

In [ ]:
to_dataframe(get_recommendations('가락국수', cosine_sim, True), get_recommendations('가락국수', distance_sim, False), bert_food_recommendation(df, dataset_cls_hidden, column, '가락국수', cosine_similarity),  bert_food_recommendation(df, dataset_cls_hidden, column, '가락국수', euclidean_distances))

,cos,euc,bert cos,bert euc
0,김치냉면,김치냉면,송이찜,가지전
1,버섯국,버섯국,민어매운탕,쇠고기산적
2,잔치국수,잔치국수,숙실과,잡산적
3,소면,소면,동태매운탕,가락국수
4,감잣국,감잣국,복(어)지리,시래기
5,쇠고기산적,쇠고기산적,잡산적,꼬치구이
6,조개국수,조개국수,시래기,복(어)지리
7,송이볶음,송이볶음,쇠고기산적,민어매운탕
8,동태찌개,동태찌개,가지전,고춧잎나물
9,비빔국수,비빔국수,가락국수,동태매운탕


In [ ]:
to_dataframe(get_recommendations('된장찌개', cosine_sim, True), get_recommendations('된장찌개', distance_sim, False), bert_food_recommendation(df, dataset_cls_hidden, column, '된장찌개', cosine_similarity),  bert_food_recommendation(df, dataset_cls_hidden, column, '된장찌개', euclidean_distances))

,cos,euc,bert cos,bert euc
0,된장국,된장국,배추속댓국,배춧국
1,청국장찌개,청국장찌개,된장국,우거짓국/우거지탕
2,우렁된장찌개,우렁된장찌개,동태섞박지,된장찌개
3,배춧국,배춧국,증편,아욱국
4,찌개,찌개,배춧국,녹두죽
5,우거짓국/우거지탕,우거짓국/우거지탕,녹두죽,동태섞박지
6,추어탕,추어탕,우거짓국/우거지탕,된장국
7,배추속댓국,배추속댓국,용봉탕,용봉탕
8,아귀탕/아구탕,아귀탕/아구탕,아욱국,증편
9,해물된장찌개,해물된장찌개,된장찌개,배추속댓국


In [ ]:
to_dataframe(get_recommendations('꼬치구이', cosine_sim, True), get_recommendations('꼬치구이', distance_sim, False), bert_food_recommendation(df, dataset_cls_hidden, column, '꼬치구이', cosine_similarity),  bert_food_recommendation(df, dataset_cls_hidden, column, '꼬치구이', euclidean_distances))

,cos,euc,bert cos,bert euc
0,송이산적,송이산적,순댓국,순댓국
1,산적,산적,깻잎나물,편포
2,쇠고기산적,쇠고기산적,버섯볶음,버섯볶음
3,비빔국수,비빔국수,잡산적,뭇국/무맑은탕
4,볶음밥,볶음밥,콩죽,꼬치구이
5,쇠고기소금구이,쇠고기소금구이,뭇국/무맑은탕,쇠고기산적
6,떡꼬치구이,떡꼬치구이,편포,콩죽
7,패주구이,패주구이,쇠고기산적,섭산적
8,곱창구이,곱창구이,섭산적,대합죽
9,자리돔구이,자리돔구이,꼬치구이,깻잎나물
